# Fundamental Analysis

In [ ]:
import pandas as pd
from vnstock_ezchart import *
from vnstock import Vnstock
ezchart = MPlot() # Khởi tạo đối tượng
# Set date range
start_date = '2024-01-01'
end_date = '2025-03-19'
interval = '1D'
symbol='REE'
stock = Vnstock().stock(symbol=symbol, source='VCI')


In [ ]:
CashFlow = stock.finance.cash_flow(period='year')


## Transpose the CF dataframe 

In [ ]:
#CashFlow
CashFlow_transposed = CashFlow.T
CashFlow_transposed.columns = CashFlow['yearReport']
# Drop the duplicate 'yearReport' row
CashFlow_transposed = CashFlow_transposed.drop('yearReport')
CashFlow_transposed.head()

# Convert the built-in visulization method to seaborn for a more polished look

In [ ]:
Ratio = stock.finance.ratio(period='year', dropna=True)


### Transpose the data frame to display on the web.

In [ ]:
Ratio_transposed = Ratio.T
Ratio_transposed.columns=Ratio_transposed.iloc[1]
Ratio_transposed = Ratio_transposed.iloc[3:]

In [ ]:
# Example: Select the 'ROE (%)' column under 'Chỉ tiêu khả năng sinh lợi'
dividend_yield = Ratio[('Chỉ tiêu khả năng sinh lợi', 'Dividend yield (%)')]
Outstanding_Shares = Ratio[('Chỉ tiêu định giá', 'Outstanding Share (Mil. Shares)')]


### Transform the balance sheet from long format to wide format. 

In [ ]:
def BS_wide(stock=None):
    """
    Transform balance sheet data into a wide format with years as columns and metrics as rows.
    
    Parameters:
    -----------
    stock : object, default=None
        The stock ticker object containing financial data.
        If None, will create a default stock object for REE from VCI source.
    
    Returns:
    --------
    pandas.DataFrame
        Transformed balance sheet with years as columns and financial metrics as rows
    """
    
    # Create default stock object if not provided
    if stock is None:
        stock = Vnstock().stock(symbol='REE', source='VCI')
    
    # Get the balance sheet data
    BS = stock.finance.balance_sheet(period='year', lang='en', dropna=True)
    
    # Transpose the DataFrame
    BS_wide = BS.T
    
    # Promote header by setting column names using the second row (index 1)
    BS_wide.columns = BS_wide.iloc[1]
    
    # Keep only the data rows (skip the first 3 rows)
    BS_wide = BS_wide.iloc[3:]
    
    return BS_wide

In [ ]:
BalanceSheet = stock.finance.balance_sheet(period='year', lang='en', dropna=True)
BalanceSheet_Transposed = BalanceSheet.T
BalanceSheet_Transposed.columns = BalanceSheet_Transposed.iloc[1]
BalanceSheet_Transposed = BalanceSheet_Transposed.iloc[3:]
BalanceSheet_Transposed.head()

### Transform Income statement from long format to wide format. 

In [ ]:
IncomeStatement = stock.finance.income_statement(period='year', lang='en', dropna=True)
IncomeStatement_Transpose= IncomeStatement.T
IncomeStatement_Transpose.columns = IncomeStatement_Transpose.iloc[1]
IncomeStatement_Transpose = IncomeStatement_Transpose.iloc[3:]
IncomeStatement_Transpose.head()

In [ ]:
# import os
# import pandas as pd

# def save_financial_statements_to_csv(balance_sheet_df, income_statement_df, cashflow_df, output_dir='./outputs'):
#     """
#     Save financial statements DataFrames to CSV files in the specified output directory.
    
#     Parameters:
#     -----------
#     balance_sheet_df : pandas.DataFrame
#         Balance Sheet DataFrame
#     income_statement_df : pandas.DataFrame
#         Income Statement DataFrame
#     cashflow_df : pandas.DataFrame
#         Cash Flow Statement DataFrame
#     output_dir : str
#         Directory path where CSV files will be saved (default: './outputs')
#     """
    
#     # Create output directory if it doesn't exist
#     try:
#         os.makedirs(output_dir, exist_ok=True)
#         print(f"Output directory '{output_dir}' is ready.")
#     except Exception as e:
#         print(f"Error creating directory: {e}")
#         return
    
#     # Dictionary of DataFrames and their corresponding filenames
#     statements = {
#         'REE_BalanceSheet_Transposed.csv': balance_sheet_df,
#         'REE_IncomeStatement_Transpose.csv': income_statement_df,
#         'REE_CashFlow_transposed.csv': cashflow_df
#     }
    
#     # Save each DataFrame to CSV
#     for filename, df in statements.items():
#         try:
#             file_path = os.path.join(output_dir, filename)
#             df.to_csv(file_path)
#             print(f"Successfully saved {filename}")
#         except Exception as e:
#             print(f"Error saving {filename}: {e}")

# # Example usage:
# # save_financial_statements_to_csv(
# #     BalanceSheet_Transposed,
# #     IncomeStatement_Transpose,
# #     CashFlow_transposed
# # )

In [ ]:
from vnstock import Vnstock
import warnings
warnings.filterwarnings("ignore")
symbol = 'REE'
company = Vnstock().stock(symbol=symbol, source='TCBS').company



# Event Impact Analysis

In [ ]:
events = stock.company.events()
dividend_events = events[events['event_list_name'] == 'Trả cổ tức bằng tiền mặt']
dividend_events = dividend_events.dropna()

print(dividend_events)


In [ ]:
# Convert from object to datetime
dividend_events['exright_date'] = pd.to_datetime(dividend_events['exright_date'])
dividend_events = dividend_events.set_index('exright_date')

In [ ]:
# Loop through each dividend event
for index, event in dividend_events.iterrows():
    # Since exright_date is now the index, we use the index directly
    exright_date = index
    
    # Calculate the time window (30 days before and after the ex-dividend date)
    exercise_start_date = exright_date - pd.Timedelta(days=30)
    exercise_end_date = exright_date + pd.Timedelta(days=30)
    
    # Convert dates to the required string format 'YYYY-MM-DD'
    exercise_start_str = exercise_start_date.strftime('%Y-%m-%d')
    exercise_end_str = exercise_end_date.strftime('%Y-%m-%d')
    
    # Get price history around the event
    event_prices = stock.quote.history(start=exercise_start_str, end=exercise_end_str, interval='1D')

In [ ]:
# Add mplfinance for quick visualization
#event_prices

In [ ]:
import plotly.graph_objects as go

# Create a candlestick chart
fig = go.Figure(data=[go.Candlestick(
    x=event_prices.index,
    open=event_prices['open'],
    high=event_prices['high'],
    low=event_prices['low'],
    close=event_prices['close']
)])

# Update layout
fig.update_layout(
    title='Candlestick Chart',
    yaxis_title='Price',
    xaxis_title='Date'
)

fig.show()

In [ ]:
# Import required libraries
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Set the style for seaborn
sns.set_style("darkgrid")
plt.figure(figsize=(14, 8))

# Create a line chart with seaborn for the price data
sns.lineplot(data=event_prices['close'], linewidth=2, label='Close Price')

# Annotate the exright dates
for date in dividend_events.index:
    if date in event_prices.index or np.datetime64(date) in event_prices.index:
        # Find the corresponding price on that date
        if date in event_prices.index:
            price = event_prices.loc[date, 'close']
        else:
            # Find the closest date if exact match not found
            closest_date = event_prices.index[event_prices.index.get_indexer([date], method='nearest')[0]]
            price = event_prices.loc[closest_date, 'close']
        
        # Annotate the exright date with a vertical line
        plt.axvline(x=date, color='red', linestyle='--', alpha=0.7)
        plt.annotate('Ex-Right Date', 
                     xy=(date, price),
                     xytext=(date, price*1.05),  # Slightly above the price
                     arrowprops=dict(facecolor='red', shrink=0.05),
                     fontsize=10,
                     ha='center')
        
        # Add markers for exercise start and end dates
        start_date = date - pd.Timedelta(days=30)
        end_date = date + pd.Timedelta(days=30)
        
        # Find prices for start and end dates
        if start_date in event_prices.index:
            start_price = event_prices.loc[start_date, 'close']
        else:
            closest_start = event_prices.index[event_prices.index.get_indexer([start_date], method='nearest')[0]]
            start_price = event_prices.loc[closest_start, 'close']
            
        if end_date in event_prices.index:
            end_price = event_prices.loc[end_date, 'close']
        else:
            closest_end = event_prices.index[event_prices.index.get_indexer([end_date], method='nearest')[0]]
            end_price = event_prices.loc[closest_end, 'close']
        
        # Annotate exercise start and end dates
        plt.axvline(x=start_date, color='green', linestyle=':', alpha=0.5)
        plt.annotate('Exercise Start', 
                     xy=(start_date, start_price),
                     xytext=(start_date, start_price*0.95),
                     arrowprops=dict(facecolor='green', shrink=0.05),
                     fontsize=9,
                     ha='center')
        
        plt.axvline(x=end_date, color='blue', linestyle=':', alpha=0.5)
        plt.annotate('Exercise End', 
                     xy=(end_date, end_price),
                     xytext=(end_date, end_price*0.95),
                     arrowprops=dict(facecolor='blue', shrink=0.05),
                     fontsize=9,
                     ha='center')

# Customize the plot
plt.title('Stock Price with Ex-Right Date Annotations', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Price', fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()
plt.legend()

# Display the plot
plt.show()

In [ ]:
CashFlow['Dividends paid']

In [ ]:
CashFlow['Levered Free Cash Flow']

In [ ]:
dividend_coverage_ratio = CashFlow['Levered Free Cash Flow'] / CashFlow['Dividends paid'].abs()

In [11]:
from vnstock import Vnstock
company = Vnstock().stock(symbol="REE",source="TCBS").company
company.dividends()

2025-05-21 17:37:30 - vnstock.common.data.data_explorer - INFO - TCBS không cung cấp thông tin danh sách. Dữ liệu tự động trả về từ VCI.


,exercise_date,cash_year,cash_dividend_percentage,issue_method
0,2025-02-27,2024,0.10,cash
1,2024-05-21,2024,0.15,share
2,2024-04-12,2023,0.10,cash
3,2023-05-19,2023,0.15,share
4,2023-02-28,2022,0.10,cash
5,2022-05-17,2022,0.15,share
6,2022-04-19,2021,0.10,cash
7,2020-03-02,2019,0.16,cash
8,2019-02-26,2018,0.18,cash
9,2018-02-28,2017,0.16,cash


In [ ]:
reciprocal_tariffs_date = '2025-04-02'
baseline_tariffs_date = '2025-04-09'
china_negotiation_date = '2025-05-12'
deal_achieved_date = '2025-05-15'